<a href="https://colab.research.google.com/github/JohnYechanJo/Novo-Nordisk_Anomaly-Detection/blob/classifier/slight_change_to_combined_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classifier Training and Result Analysis by Synthetic Data Ratio slight change version

In [ ]:
def train_classifier_with_ratios():
    normal_dataset = torch.load('pre-trained_dataset.pt')
    synthetic_dataset = torch.load('synthetic_cnv_dataset.pt')

    normal_cnv_indices = normal_dataset['label'] == 0
    normal_normal_indices = normal_dataset['label'] == 1
    normal_cnv_data = {
        'data_0': normal_dataset['data_0'][normal_cnv_indices],
        'data_1': normal_dataset['data_1'][normal_cnv_indices],
        'data_2': normal_dataset['data_2'][normal_cnv_indices],
        'data_3': normal_dataset['data_3'][normal_cnv_indices],
        'label': normal_dataset['label'][normal_cnv_indices]
    }
    normal_normal_data = {
        'data_0': normal_dataset['data_0'][normal_normal_indices],
        'data_1': normal_dataset['data_1'][normal_normal_indices],
        'data_2': normal_dataset['data_2'][normal_normal_indices],
        'data_3': normal_dataset['data_3'][normal_normal_indices],
        'label': normal_dataset['label'][normal_normal_indices]
    }
    synthetic_cnv_data = synthetic_dataset

    ratios = np.arange(0, 1.1, 0.1)
    results = []

    for ratio in ratios:
        print(f"\nTraining with Synthetic Ratio: {ratio*100:.0f}%")
        num_synthetic_batch = int(64 * ratio)
        num_normal_cnv_batch = 64 - num_synthetic_batch
        mixed_data_0 = []
        mixed_data_1 = []
        mixed_data_2 = []
        mixed_data_3 = []
        labels_list = []
        for i in range(batch_num):
            labels = torch.cat([torch.zeros(half_batch, dtype=torch.long), torch.ones(half_batch, dtype=torch.long)], dim=0)
            s_cnv_0 = synthetic_cnv_data['data_0'][i*num_synthetic_batch:(i+1)*num_synthetic_batch]
            s_cnv_1 = synthetic_cnv_data['data_1'][i*num_synthetic_batch:(i+1)*num_synthetic_batch]
            s_cnv_2 = synthetic_cnv_data['data_2'][i*num_synthetic_batch:(i+1)*num_synthetic_batch]
            s_cnv_3 = synthetic_cnv_data['data_3'][i*num_synthetic_batch:(i+1)*num_synthetic_batch]
            n_cnv_0 = normal_cnv_data['data_0'][i*num_normal_cnv_batch:(i+1)*num_normal_cnv_batch]
            n_cnv_1 = normal_cnv_data['data_1'][i*num_normal_cnv_batch:(i+1)*num_normal_cnv_batch]
            n_cnv_2 = normal_cnv_data['data_2'][i*num_normal_cnv_batch:(i+1)*num_normal_cnv_batch]
            n_cnv_3 = normal_cnv_data['data_3'][i*num_normal_cnv_batch:(i+1)*num_normal_cnv_batch]
            n_normal_0 = normal_normal_data['data_0'][i*half_batch:(i+1)*half_batch]
            n_normal_1 = normal_normal_data['data_1'][i*half_batch:(i+1)*half_batch]
            n_normal_2 = normal_normal_data['data_2'][i*half_batch:(i+1)*half_batch]
            n_normal_3 = normal_normal_data['data_3'][i*half_batch:(i+1)*half_batch]
            indices = torch.randperm(batch_size)
            compose_batch_0 = torch.cat([s_cnv_0,n_cnv_0,n_normal_0],dim=0)
            compose_batch_1 = torch.cat([s_cnv_1,n_cnv_1,n_normal_1],dim=0)
            compose_batch_2 = torch.cat([s_cnv_2,n_cnv_2,n_normal_2],dim=0)
            compose_batch_3 = torch.cat([s_cnv_3,n_cnv_3,n_normal_3],dim=0)
            # 128 per batch balance and shuffle
            mixed_data_0.append(compose_batch_0[indices])
            mixed_data_1.append(compose_batch_1[indices])
            mixed_data_2.append(compose_batch_2[indices])
            mixed_data_3.append(compose_batch_3[indices])
            labels_list.append(labels[indices])

        mixed_data_0 = torch.cat(mixed_data_0, dim=0)
        mixed_data_1 = torch.cat(mixed_data_1, dim=0)
        mixed_data_2 = torch.cat(mixed_data_2, dim=0)
        mixed_data_3 = torch.cat(mixed_data_3, dim=0)
        mixed_labels = torch.cat(labels_list,dim=0)

        x_train = torch.arange(0, 1024)
        x_val = torch.arange(1024, 1152)
        x_test = torch.arange(1152, 1280)
        y_train = mixed_labels[:1024]
        y_val = mixed_labels[1024:1152]
        y_test = mixed_labels[1152:1280]

        # 임베딩 저장
        if os.path.exists('pre-trained_dataset.pt'):
            os.remove('pre-trained_dataset.pt')
        torch.save({
            'data_0': mixed_data_0,
            'data_1': mixed_data_1,
            'data_2': mixed_data_2,
            'data_3': mixed_data_3,
            'label': mixed_labels
        }, 'pre-trained_dataset.pt')

        model = Classifier()
        res = train_and_test(model, x_train, y_train, x_val, y_val, x_test, y_test)

        results.append({
            'ratio': ratio,
            'accuracy': res['accuracy'],
            'f1_score': res['macro avg']['f1-score'],
            'precision': res['macro avg']['precision'],
            'recall': res['macro avg']['recall']
        })
        clear_memory()

    print("\nResults Summary:")
    for res in results:
        print(f"Ratio: {res['ratio']*100:.0f}% | Accuracy: {res['accuracy']:.4f} | F1 Score: {res['f1_score']:.4f} | Precision: {res['precision']:.4f} | Recall: {res['recall']:.4f}")

    best_result = max(results, key=lambda x: x['accuracy'])
    print(f"\nBest Ratio: {best_result['ratio']*100:.0f}%")
    print(f"Accuracy: {best_result['accuracy']:.4f}")
    print(f"F1 Score: {best_result['f1_score']:.4f}")
    print(f"Precision: {best_result['precision']:.4f}")
    print(f"Recall: {best_result['recall']:.4f}")

    pd.DataFrame(results).to_csv('classifier_results.csv', index=False)
    print("Results saved to classifier_results.csv")

train_classifier_with_ratios()